# IMPORTS

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split

from scipy import ndimage
from skimage.io import imread
from skimage import color
from skimage.transform import resize
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import time
import cv2
from PIL import Image
%matplotlib inline
import scipy.misc
from sklearn.svm import SVC

from imblearn import under_sampling, over_sampling


from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier
from sklearn import preprocessing

In [16]:
feat_df = pd.read_csv('data/features.csv', sep=',')


def shape_asymetry(idRep):
    t = time.time()
    m=0
    for x in idRep:
        m+=1
        filename = 'data/im_resized/{}_resized.jpg'.format(x)
        filename_S = 'data/im_resized/{}_Segmentation_resized.jpg'.format(x)
        image = imread(filename)
        image_S = imread(filename_S)
        image_S = image_S.astype(bool)
        
        #On convertit l'image en niveau de gris
        gray_image = color.rgb2gray(image)*255
        n,p = gray_image.shape
        i_center, j_center = ndimage.measurements.center_of_mass(image_S)
        
        
        masked_im = np.array(gray_image*image_S, np.int32)
        
        averageDS = 100
        
        
        padX = [p - int(j_center), int(j_center)]
        padY = [n - int(i_center), int(i_center)]
        imgP = np.pad(masked_im, [padY, padX], 'constant')
        
        for angle in [10*k for k in range(18)]:
            
         
            rotated_image = ndimage.rotate(imgP, angle, output=np.int32 ,reshape=False)
           
            rotated_image = np.clip(rotated_image, 0, 255)
            
            
            nbis, pbis = rotated_image.shape
            
            
            DS1 = np.sum(np.abs(rotated_image[:,:pbis//2]-np.fliplr(rotated_image[:,pbis//2:pbis])))
            DS2 = np.sum(np.abs(rotated_image[:nbis//2,:]-np.flipud(rotated_image[nbis//2:nbis, :])))
            
            
            DS1 /= np.sum(np.sum(image_S*255))
            DS2 /= np.sum(np.sum(image_S*255))
            if((DS1+DS2)/2 < averageDS):
                minDS1 = DS1
                minDS2 = DS2
                averageDS = (DS1+DS2)/2
        
        if(m%50 ==0):
            print(time.time()-t)
            print(m," : tour 1")
        feat_df.loc[feat_df['ImageId'] == x, 'f1'] = minDS1
        feat_df.loc[feat_df['ImageId'] == x, 'f2'] = minDS2
    print(time.time()-t)
    feat_df.to_csv('data/features.csv', index=None, sep=',', mode='w')
feat_df = feat_df[feat_df['ImageId']!='IM_000720']
#shape_asymetry(feat_df['ImageId'].values)

In [11]:
feat_df['f1'] = 255*feat_df['f1']

In [13]:
feat_df['f2'] = 255*feat_df['f2']

In [14]:
feat_df.to_csv('data/features.csv', index=None, sep=',', mode='w')